In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
dataDirectory = "imgs/" # folder that contains the dataset

In [ ]:
img_size = 224 #imagenetv2 input size
classes = ["0", "1", "2", "3", "4", "5", "6"] #list of classes
trainingData = []

In [ ]:
def create_training_data():
    for category in classes:
        img_path = os.path.join(dataDirectory, category)
        for img in os.listdir(img_path):
            try:
                img_array = cv2.imread(os.path.join(img_path, img))
                resized_img = cv2.resize(img_array, (img_size, img_size))
                trainingData.append([resized_img, int(category)])
            except Exception as e :
                pass

In [ ]:
create_training_data()

In [ ]:
import random
random.shuffle(trainingData)

In [ ]:
X, y = [], []

for img, label in trainingData:
    X.append(img)
    y.append(label)

In [ ]:
X = np.array(X).reshape(-1, img_size, img_size, 3)
X = X / 255.0 #normalize the data

y = np.array(y)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

In [ ]:
input = model.layers[0].input
output = model.layers[-2].output

In [ ]:
finalOutput = layers.Dense(128)(output) # adding dense layer after GlobalAveragePooling layer
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(64)(finalOutput_)
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(7, activation='softmax')(finalOutput_) # we are working with 7 classes

In [ ]:
model = keras.Model(inputs= input, outputs= finalOutput)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=20)

In [ ]:
model.save("final_model.h5")